In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import re

import numpy as np
import pandas as pd

import kraft

In [ ]:
fcs_file_paths = []

for directory_path, directory_names, file_names in os.walk(
    "/Users/kwatme/Downloads/fcs"
):

    for file_name in file_names:

        if file_name.endswith(".fcs"):

            fcs_file_paths.append("{}/{}".format(directory_path, file_name))

fcs_file_paths = sorted(fcs_file_paths)

fcs_file_paths

In [ ]:
pnss = []

for i, fcs_file_path in enumerate(fcs_file_paths):

    print(i + 1, fcs_file_path)

    pnss += kraft.read_fcs(fcs_file_path)[1].columns.tolist()

In [ ]:
pns_name = {
    "time": "cell_time",
    "cell_length": "cell_size",
    "event_length": "cell_size",
    "bc1": "barcode_1",
    "bc2": "barcode_2",
    "bc3": "barcode_3",
    "bc4": "barcode_4",
    "bc5": "barcode_5",
    "bc6": "barcode_6",
    "dna1": "notprotein_dna1",
    "dna2": "notprotein_dna2",
    "viability": "notprotein_cisplatin",
    "cisplatin_viability": "notprotein_cisplatin",
    "cd10": "protein_cd10",
    "cd123": "protein_cd123",
    "cd127": "protein_cd127",
    "cd132": "protein_cd132",
    "cd179a": "protein_cd179a",
    "cd179b": "protein_cd179b",
    "cd19": "protein_cd19",
    "cd20": "protein_cd20",
    "cd22": "protein_cd22",
    "cd235-61": "protein_cd23561",
    "cd235_61": "protein_cd23561",
    "cd24": "protein_cd24",
    "cd3": "protein_cd3",
    "cd33-cd16": "protein_cd33cd16",
    "cd34": "protein_cd34",
    "cd38": "protein_cd38",
    "cd43": "protein_cd43",
    "cd45": "protein_cd45",
    "cd58": "protein_cd58",
    "cd79b": "protein_cd79b",
    "ce140": "protein_ce140",
    "cparp": "protein_cparp",
    "fitc_myeloid": "protein_fitcmyeloid",
    "gd157": "protein_gd157",
    "hla-dr": "protein_hladr",
    "hladr": "protein_hladr",
    "i127": "protein_i127",
    "igmi": "protein_igmi",
    "igms": "protein_igms",
    "kappa-lambda": "protein_kappalambda",
    "kappa_lambda": "protein_kappalambda",
    "ki67": "protein_ki67",
    "p4ebp1": "protein_p4ebp1",
    "pakt": "protein_pakt",
    "pax5": "protein_pax5",
    "pcreb": "protein_pcreb",
    "perk": "protein_perk",
    "pikaros": "protein_pikaros",
    "pplcg1-2": "protein_pplcg12",
    "pplcg1_2": "protein_pplcg12",
    "ps6": "protein_ps6",
    "pstat5": "protein_pstat5",
    "psyk": "protein_psyk",
    "rag1": "protein_rag1",
    "tdt": "protein_tdt",
    "tikaros": "protein_tikaros",
    "tslpr": "protein_tslpr",
}

In [ ]:
new_pns_name = {}

for pns in pnss:

    pns = pns.lower()

    if pns not in pns_name:

        new_pns_name[pns] = re.sub("[^A-Za-z0-9]+", "", pns)

for new_pns, name in new_pns_name.items():

    print('"{}": "p_{}",'.format(new_pns, name))

In [ ]:
n_print = 3

for fcs_file_path in fcs_file_paths:

    if not re.search("Healthy[1]+_Basal", fcs_file_path):

        continue

    print(fcs_file_path)

    text_dict, information_x_pns, pns_x_cell = kraft.read_fcs(fcs_file_path)

    # Save information_x_pns

    # Save pns_x_cell

    parameters = pd.Index(
        information_x_pns.columns.str.lower().map(pns_name), name="Parameter"
    )

    parameter_is_good = ~parameters.isna()

    good_parameters = parameters[parameter_is_good]

    information_x_parameter = information_x_pns.loc[:, parameter_is_good]

    information_x_parameter.columns = good_parameters

    # Save information_x_parameter

    parameter_x_cell = pns_x_cell.loc[parameter_is_good]

    parameter_x_cell.index = good_parameters

    # Save parameter_x_cell

    protein_x_cell = parameter_x_cell.loc[
        parameter_x_cell.index.str.startswith("protein_")
    ]

    protein_x_cell = np.ceil(protein_x_cell.clip(lower=0).astype(int))

    print(protein_x_cell.shape)

    # Save protein_x_cell

In [ ]:
for protein, values in protein_x_cell.iterrows():

    kraft.plot_histogram((values,))

    raise